# Getting started with Azure ML Data Prep SDK
Copyright (c) Microsoft Corporation. All rights reserved.<br>
Licensed under the MIT License.


Wonder how you can make the most of the Azure ML Data Prep SDK? In this "Getting Started" guide, we'll demonstrate how to do your normal data wrangling with this SDK and showcase a few highlights that make this SDK shine. Using a sample of this [Kaggle crime dataset](https://www.kaggle.com/currie32/crimes-in-chicago/home) as an example, we'll cover how to:

* [Read in data](#Read-in-data)
* [Profile your data](#Profile-your-data)
* [Append rows](#Append-rows)
* [Apply common data science transforms](#Apply-common-data-science-transforms)
    * [Summarize](#Summarize)
    * [Join](#Join)
    * [Filter](#Filter)
    * [Replace](#Replace)
* [Consume your cleaned dataset](#Consume-your-cleaned-dataset)
* [Explore advanced features](#Explore-advanced-features)


In [1]:
from IPython.display import display
from os import path
from tempfile import mkdtemp

import pandas as pd
import azureml.dataprep as dprep

# Paths for datasets
file_crime_dirty  = '../../data/crime-dirty.csv'
file_crime_spring = '../../data/crime-spring.csv'
file_crime_winter = '../../data/crime-winter.csv'
file_aldermen     = '../../data/chicago-aldermen-2015.csv'

# Seed
RAND_SEED = 7251

## Read in data

Azure ML Data Prep supports many different file reading formats (i.e. CSV, Excel, Parquet) and the ability to infer column types automatically. To see how powerful the `auto_read_file` capability is, let's take a peek at the `dirty-crime.csv`:

In [2]:
dprep.read_csv(path=file_crime_dirty).head(7)

,File updated 11/2/2018,Column2
0,,None
1,,None
2,,None
3,ID|Case Number|Date|Block|IUCR|Primary Type|De...,None
4,10140490|HY329907|07/05/2015 11:50:00 PM|050XX...,-87.800174996)
5,10139776|HY329265|07/05/2015 11:30:00 PM|011XX...,-87.65955018)
6,10140270|HY329253|07/05/2015 11:20:00 PM|121XX...,None


A common occurrence in many datasets is to have a column of values with commas; in our case, the last column represents location in the form of longitude-latitude pair. The default CSV reader interprets this comma as a delimiter and thus splits the data into two columns. Furthermore, it incorrectly reads in the header as the column name. Normally, we would need to `skip` the header and specify the delimiter as `|`, but our `auto_read_file` eliminates that work:

In [3]:
crime_dirty = dprep.auto_read_file(path=file_crime_dirty)

crime_dirty.head(5)

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10140490.0,HY329907,07/05/2015 11:50:00 PM,050XX N NEWLAND AVE,820.0,THEFT,$500 AND UNDER,STREET,False,False,...,41.0,10.0,06,1129230.0,1933315.0,2015.0,07/12/2015 12:42:46 PM,41.973309,-87.800175,"(41.973309466, -87.800174996)"
1,10139776.0,HY329265,07/05/2015 11:30:00 PM,011XX W MORSE AVE,460.0,BATTERY,SIMPLE,STREET,False,True,...,49.0,1.0,08B,1167370.0,1946271.0,2015.0,07/12/2015 12:42:46 PM,42.008124,-87.659550,"(42.008124017, -87.65955018)"
2,10140270.0,HY329253,07/05/2015 11:20:00 PM,121XX S FRONT AVE,486.0,BATTERY,DOMESTIC BATTERY SIMPLE,STREET,False,True,...,9.0,53.0,08B,NaN,NaN,2015.0,07/12/2015 12:42:46 PM,NaN,NaN,
3,10139885.0,HY329308,07/05/2015 11:19:00 PM,051XX W DIVISION ST,610.0,BURGLARY,FORCIBLE ENTRY,SMALL RETAIL STORE,False,False,...,37.0,25.0,05,1141721.0,1907465.0,2015.0,07/12/2015 12:42:46 PM,41.902152,-87.754883,"(41.902152027, -87.754883404)"
4,10140379.0,HY329556,07/05/2015 11:00:00 PM,012XX W LAKE ST,930.0,MOTOR VEHICLE THEFT,THEFT/RECOVERY: AUTOMOBILE,STREET,False,False,...,27.0,28.0,07,1168413.0,1901632.0,2015.0,07/12/2015 12:42:46 PM,41.885610,-87.657009,"(41.885610142, -87.657008701)"


__Advanced features:__ if you'd like to specify the file type and adjust how you want to read files in, you can see the list of our specialized file readers and how to use them [here](../../how-to-guides/data-ingestion.ipynb).

## Profile your data

Let's understand what our data looks like. Azure ML Data Prep facilitates this process by offering data profiles that help us glimpse into column types and column summary statistics. Notice that our auto file reader automatically guessed the column type:

In [4]:
crime_dirty.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent missing,Error Count,Empty count,0.1% Quantile,1% Quantile,5% Quantile,25% Quantile,50% Quantile,75% Quantile,95% Quantile,99% Quantile,99.9% Quantile,Mean,Standard Deviation,Variance,Skewness,Kurtosis
ID,FieldType.DECIMAL,1.01397e+07,1.01409e+07,10.0,0.0,10.0,0.0,0.0,0.0,1.01397e+07,1.01397e+07,1.01397e+07,1.01398e+07,1.01398e+07,1.01404e+07,1.01409e+07,1.01409e+07,1.01409e+07,1.01401e+07,409.806,167941,0.688352,-1.15364
Case Number,FieldType.STRING,HY329177,HY330421,10.0,0.0,10.0,0.0,0.0,0.0,,,,,,,,,,,,,,
Date,FieldType.STRING,07/05/2015 10:10:00 PM,07/05/2015 11:50:00 PM,10.0,0.0,10.0,0.0,0.0,0.0,,,,,,,,,,,,,,
Block,FieldType.STRING,011XX W MORSE AVE,121XX S FRONT AVE,10.0,0.0,10.0,0.0,0.0,0.0,,,,,,,,,,,,,,
IUCR,FieldType.DECIMAL,460,1811,10.0,0.0,10.0,0.0,0.0,0.0,460,473,460,610,975,1320,1811,1811,1811,1008.7,435.056,189273,0.27388,-1.23243
Primary Type,FieldType.STRING,ARSON,THEFT,10.0,0.0,10.0,0.0,0.0,0.0,,,,,,,,,,,,,,
Description,FieldType.STRING,$500 AND UNDER,TO VEHICLE,10.0,0.0,10.0,0.0,0.0,0.0,,,,,,,,,,,,,,
Location Description,FieldType.STRING,ALLEY,VEHICLE NON-COMMERCIAL,10.0,0.0,10.0,0.0,0.0,0.0,,,,,,,,,,,,,,
Arrest,FieldType.BOOLEAN,False,True,10.0,0.0,10.0,0.0,0.0,0.0,,,,,,,,,,,,,,
Domestic,FieldType.BOOLEAN,False,True,10.0,0.0,10.0,0.0,0.0,0.0,,,,,,,,,,,,,,


## Append rows

What if your data is split across multiple files? We support the ability to append multiple datasets column-wise and row-wise. Here, we demonstrate how you can coalesce datasets row-wise:

In [5]:
# Datasets with the same schema as crime_dirty
crime_winter = dprep.auto_read_file(path=file_crime_winter)
crime_spring = dprep.auto_read_file(path=file_crime_spring)

In [6]:
crime = (crime_dirty.append_rows(dataflows=[crime_winter, crime_spring]))

crime.take_sample(probability=0.25, seed=RAND_SEED).head(5)

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10140270.0,HY329253,07/05/2015 11:20:00 PM,121XX S FRONT AVE,486.0,BATTERY,DOMESTIC BATTERY SIMPLE,STREET,False,True,...,9.0,53.0,08B,NaN,NaN,2015.0,07/12/2015 12:42:46 PM,NaN,NaN,
1,10374287.0,HZ110730,1/10/2016 11:50,043XX W ARMITAGE AVE,5002.0,OTHER OFFENSE,OTHER VEHICLE OFFENSE,STREET,False,False,...,30.0,20.0,26,1146917.0,1912931.0,2016.0,2016-06-07T15:55:00.000000,41.917054,-87.735658,"(41.917053561, -87.735657637)"
2,10375178.0,HZ110832,1/10/2016 14:20,057XX S KEDZIE AVE,460.0,BATTERY,SIMPLE,RESTAURANT,False,False,...,14.0,63.0,08B,1156029.0,1866379.0,2016.0,2016-02-04T15:44:00.000000,41.789131,-87.703435,"(41.78913051, -87.703434602)"
3,10535059.0,HZ278872,2016-04-15T04:30:00.000000,004XX S KILBOURN AVE,810.0,THEFT,OVER $500,RESIDENCE,False,False,...,24.0,26.0,6,NaN,NaN,2016.0,2016-05-25T15:59:00.000000,NaN,NaN,


__Advanced features:__ you can learn how to append column-wise and how to deal with appending data with different schemas [here](../../how-to-guides/append-columns-and-rows.ipynb).

## Apply common data science transforms

Azure ML Data Prep supports almost all common data science transforms found in other industry-standard data science libraries. Here, we'll explore the ability to `summarize`, `join`, `filter`, and `replace`. 

__Advanced features:__
* We also provide "smart" transforms not found in pandas that use machine learning to [derive new columns](../../how-to-guides/derive-column-by-example.ipynb), [split columns](../../how-to-guides/split-column-by-example.ipynb), and [fuzzy grouping](../../how-to-guides/fuzzy-group.ipynb).
* Finally, we also help featurize your dataset to prepare it for machine learning; learn more about our featurizers like [one-hot encoder](../../how-to-guides/one-hot-encoder.ipynb), [label encoder](../../how-to-guides/label-encoder.ipynb), [min-max scaler](../../how-to-guides/min-max-scaler.ipynb), and [random (train-test) split](../../how-to-guides/random-split.ipynb).
* Our complete list of example Notebooks for transforms can be found in our [How-to Guides](../../how-to-guides).

### Summarize

Let's see which wards had the most crimes in our sample dataset:

In [7]:
crime_summary = (crime
    .summarize(
        summary_columns=[
            dprep.SummaryColumnsValue(
                column_id='ID', 
                summary_column_name='total_ward_crimes', 
                summary_function=dprep.SummaryFunction.COUNT
            )
        ],
        group_by_columns=['Ward']
    )
)

(crime_summary
     .sort(sort_order=[('total_ward_crimes', True)])
     .head(5)
)

,Ward,total_ward_crimes
0,9.0,3
1,41.0,2
2,16.0,2
3,24.0,2
4,14.0,2


### Join

Let's annotate each observation with more information about the ward where the crime occurred. Let's do so by joining `crime` with a dataset which lists the current aldermen for each ward:

In [8]:
aldermen = dprep.auto_read_file(path=file_aldermen)

aldermen.head(5)

,Ward,Name,Took Office,Party
0,1.0,Proco Joe Moreno,2010*,Dem
1,2.0,Brian Hopkins,2015,Dem
2,3.0,Pat Dowell,2007,Dem
3,4.0,Sophia King,2016*,Dem
4,5.0,Leslie Hairston,1999,Dem


In [9]:
crime.join(
    left_dataflow=crime,
    right_dataflow=aldermen,
    join_key_pairs=[
        ('Ward', 'Ward')
    ]
).head(5)

,l_ID,l_Case Number,l_Date,l_Block,l_IUCR,l_Primary Type,l_Description,l_Location Description,l_Arrest,l_Domestic,...,l_Y Coordinate,l_Year,l_Updated On,l_Latitude,l_Longitude,l_Location,r_Ward,r_Name,r_Took Office,r_Party
0,10140490.0,HY329907,07/05/2015 11:50:00 PM,050XX N NEWLAND AVE,820.0,THEFT,$500 AND UNDER,STREET,False,False,...,1933315.0,2015.0,07/12/2015 12:42:46 PM,41.973309,-87.800175,"(41.973309466, -87.800174996)",41.0,Anthony Napolitano,2015,Rep
1,10400131.0,HZ136171,1/10/2016 18:00,0000X W TERMINAL ST,810.0,THEFT,OVER $500,AIRPORT BUILDING NON-TERMINAL - SECURE AREA,False,False,...,NaN,2016.0,2016-02-02T15:58:00.000000,NaN,NaN,,41.0,Anthony Napolitano,2015,Rep
2,10139776.0,HY329265,07/05/2015 11:30:00 PM,011XX W MORSE AVE,460.0,BATTERY,SIMPLE,STREET,False,True,...,1946271.0,2015.0,07/12/2015 12:42:46 PM,42.008124,-87.659550,"(42.008124017, -87.65955018)",49.0,Joe Moore,1991,Dem
3,10140270.0,HY329253,07/05/2015 11:20:00 PM,121XX S FRONT AVE,486.0,BATTERY,DOMESTIC BATTERY SIMPLE,STREET,False,True,...,NaN,2015.0,07/12/2015 12:42:46 PM,NaN,NaN,,9.0,Anthony Beale,1999,Dem
4,10498554.0,HZ239907,2016-04-15T23:56:00.000000,007XX E 111TH ST,1153.0,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,OTHER,False,False,...,1831503.0,2016.0,2016-05-11T15:48:00.000000,41.692834,-87.604319,"(41.692833841, -87.60431945)",9.0,Anthony Beale,1999,Dem


__Advanced features:__ [Learn more](../../how-to-guides/join.ipynb) about how you can do all variants of `join`, like inner-, left-, right-, anti-, and semi-joins.

### Filter

Let's look at theft crimes:

In [10]:
theft = crime.filter(crime['Primary Type'] == 'THEFT')

theft.head(5)

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10140490.0,HY329907,07/05/2015 11:50:00 PM,050XX N NEWLAND AVE,820.0,THEFT,$500 AND UNDER,STREET,False,False,...,41.0,10.0,06,1129230.0,1933315.0,2015.0,07/12/2015 12:42:46 PM,41.973309,-87.800175,"(41.973309466, -87.800174996)"
1,10374720.0,HZ110836,1/10/2016 7:30,079XX S RHODES AVE,890.0,THEFT,FROM BUILDING,OTHER,False,False,...,6.0,44.0,6,1181279.0,1852568.0,2016.0,2016-02-04T15:44:00.000000,41.750687,-87.611277,"(41.75068679, -87.611276811)"
2,10400131.0,HZ136171,1/10/2016 18:00,0000X W TERMINAL ST,810.0,THEFT,OVER $500,AIRPORT BUILDING NON-TERMINAL - SECURE AREA,False,False,...,41.0,76.0,6,NaN,NaN,2016.0,2016-02-02T15:58:00.000000,NaN,NaN,
3,10516598.0,HZ258664,2016-04-15T17:00:00.000000,082XX S MARSHFIELD AVE,890.0,THEFT,FROM BUILDING,RESIDENCE,False,False,...,21.0,71.0,6,1166776.0,1850053.0,2016.0,2016-05-12T15:48:00.000000,41.744107,-87.664494,"(41.744106973, -87.664494285)"
4,10534446.0,HZ277630,2016-04-15T10:00:00.000000,055XX N KEDZIE AVE,890.0,THEFT,FROM BUILDING,"SCHOOL, PUBLIC, BUILDING",False,False,...,40.0,13.0,6,NaN,NaN,2016.0,2016-05-25T15:59:00.000000,NaN,NaN,


### Replace

Notice that our `theft` dataset has empty strings in column `Location`. Let's replace those with a missing value:

In [11]:
theft_replaced = (theft
    .replace_na(
        columns=['Location'], 
        use_empty_string_as_na=True
    )
)

theft_replaced.head(5)

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10140490.0,HY329907,07/05/2015 11:50:00 PM,050XX N NEWLAND AVE,820.0,THEFT,$500 AND UNDER,STREET,False,False,...,41.0,10.0,06,1129230.0,1933315.0,2015.0,07/12/2015 12:42:46 PM,41.973309,-87.800175,"(41.973309466, -87.800174996)"
1,10374720.0,HZ110836,1/10/2016 7:30,079XX S RHODES AVE,890.0,THEFT,FROM BUILDING,OTHER,False,False,...,6.0,44.0,6,1181279.0,1852568.0,2016.0,2016-02-04T15:44:00.000000,41.750687,-87.611277,"(41.75068679, -87.611276811)"
2,10400131.0,HZ136171,1/10/2016 18:00,0000X W TERMINAL ST,810.0,THEFT,OVER $500,AIRPORT BUILDING NON-TERMINAL - SECURE AREA,False,False,...,41.0,76.0,6,NaN,NaN,2016.0,2016-02-02T15:58:00.000000,NaN,NaN,None
3,10516598.0,HZ258664,2016-04-15T17:00:00.000000,082XX S MARSHFIELD AVE,890.0,THEFT,FROM BUILDING,RESIDENCE,False,False,...,21.0,71.0,6,1166776.0,1850053.0,2016.0,2016-05-12T15:48:00.000000,41.744107,-87.664494,"(41.744106973, -87.664494285)"
4,10534446.0,HZ277630,2016-04-15T10:00:00.000000,055XX N KEDZIE AVE,890.0,THEFT,FROM BUILDING,"SCHOOL, PUBLIC, BUILDING",False,False,...,40.0,13.0,6,NaN,NaN,2016.0,2016-05-25T15:59:00.000000,NaN,NaN,None


__Advanced features:__ [Learn more](../../how-to-guides/replace-fill-error.ipynb) about more advanced `replace` and `fill` capabilities.

## Consume your cleaned dataset

Azure ML Data Prep allows you to "choose your own adventure" once you're done wrangling. You can:

1. Write to a pandas dataframe
2. Execute on Spark
3. Consume directly in Azure Machine Learning models

In this quickstart guide, we'll show how you can export to a pandas dataframe.

__Advanced features:__ 
* One of the beautiful features of Azure ML Data Prep is that you only need to write your code once and choose whether to scale up or out.
* You can directly consume your new DataFlow in model builders like Azure Machine Learning's [automated machine learning](https://github.com/Azure/MachineLearningNotebooks/blob/master/automl/13a.auto-ml-dataprep.ipynb).

In [12]:
theft_replaced.to_pandas_dataframe()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10140490.0,HY329907,07/05/2015 11:50:00 PM,050XX N NEWLAND AVE,820.0,THEFT,$500 AND UNDER,STREET,False,False,...,41.0,10.0,06,1129230.0,1933315.0,2015.0,07/12/2015 12:42:46 PM,41.973309,-87.800175,"(41.973309466, -87.800174996)"
1,10374720.0,HZ110836,1/10/2016 7:30,079XX S RHODES AVE,890.0,THEFT,FROM BUILDING,OTHER,False,False,...,6.0,44.0,6,1181279.0,1852568.0,2016.0,2016-02-04T15:44:00.000000,41.750687,-87.611277,"(41.75068679, -87.611276811)"
2,10400131.0,HZ136171,1/10/2016 18:00,0000X W TERMINAL ST,810.0,THEFT,OVER $500,AIRPORT BUILDING NON-TERMINAL - SECURE AREA,False,False,...,41.0,76.0,6,NaN,NaN,2016.0,2016-02-02T15:58:00.000000,NaN,NaN,None
3,10516598.0,HZ258664,2016-04-15T17:00:00.000000,082XX S MARSHFIELD AVE,890.0,THEFT,FROM BUILDING,RESIDENCE,False,False,...,21.0,71.0,6,1166776.0,1850053.0,2016.0,2016-05-12T15:48:00.000000,41.744107,-87.664494,"(41.744106973, -87.664494285)"
4,10534446.0,HZ277630,2016-04-15T10:00:00.000000,055XX N KEDZIE AVE,890.0,THEFT,FROM BUILDING,"SCHOOL, PUBLIC, BUILDING",False,False,...,40.0,13.0,6,NaN,NaN,2016.0,2016-05-25T15:59:00.000000,NaN,NaN,None
5,10535059.0,HZ278872,2016-04-15T04:30:00.000000,004XX S KILBOURN AVE,810.0,THEFT,OVER $500,RESIDENCE,False,False,...,24.0,26.0,6,NaN,NaN,2016.0,2016-05-25T15:59:00.000000,NaN,NaN,None


## Explore advanced features

Congratulations on finishing your introduction to the Azure ML Data Prep SDK! If you'd like more detailed tutorials on how to construct machine learning datasets or dive deeper into all of its functionality, you can find more information in our detailed notebooks [here](https://github.com/Microsoft/PendletonDocs). There, we cover topics including how to:

* [Cache your Dataflow to speed up your iterations](../../how-to-guides/cache.ipynb)
* [Add your custom Python transforms](../../how-to-guides/custom-python-transforms.ipynb)
* [Impute missing values](../../how-to-guides/impute-missing-values.ipynb)
* [Sample your data](../../how-to-guides/subsetting-sampling.ipynb)
* [Reference and link between Dataflows](../../how-to-guides/join.ipynb)